# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 9:53AM,254898,10,Enova-11198,Emerginnova,Released
1,Jan 12 2023 9:44AM,254894,10,0086295527,ISDIN Corporation,Released
2,Jan 12 2023 9:44AM,254894,10,0086295515,ISDIN Corporation,Released
3,Jan 12 2023 9:44AM,254894,10,0086295518,ISDIN Corporation,Released
4,Jan 12 2023 9:44AM,254894,10,0086295516,ISDIN Corporation,Released
5,Jan 12 2023 9:44AM,254894,10,0086295526,ISDIN Corporation,Released
6,Jan 12 2023 9:44AM,254893,10,0086295524,ISDIN Corporation,Released
7,Jan 12 2023 9:44AM,254893,10,0086295519,ISDIN Corporation,Released
8,Jan 12 2023 9:44AM,254893,10,0086295531,ISDIN Corporation,Released
9,Jan 12 2023 9:44AM,254893,10,0086295877,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,254890,Released,1
40,254891,Released,4
41,254893,Released,4
42,254894,Released,5
43,254898,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254890,NaN,3.0,1.0
254891,NaN,NaN,4.0
254893,NaN,NaN,4.0
254894,NaN,NaN,5.0
254898,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254770,7.0,3.0,2.0
254799,0.0,1.0,0.0
254801,0.0,0.0,1.0
254806,0.0,0.0,1.0
254810,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254770,7,3,2
254799,0,1,0
254801,0,0,1
254806,0,0,1
254810,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254770,7,3,2
1,254799,0,1,0
2,254801,0,0,1
3,254806,0,0,1
4,254810,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254770,7,3,2
1,254799,,1,
2,254801,,,1
3,254806,,,1
4,254810,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 9:53AM,254898,10,Emerginnova
1,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation
6,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation
10,Jan 12 2023 9:27AM,254891,20,"Exact-Rx, Inc."
14,Jan 12 2023 9:18AM,254890,16,Sartorius Lab Products and Service
18,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc."
20,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation
44,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation
77,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation
110,Jan 12 2023 9:01AM,254886,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 9:53AM,254898,10,Emerginnova,,,1
1,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,,,5
2,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,,,4
3,Jan 12 2023 9:27AM,254891,20,"Exact-Rx, Inc.",,,4
4,Jan 12 2023 9:18AM,254890,16,Sartorius Lab Products and Service,,3,1
5,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",,2,
6,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,,,24
7,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,,,33
8,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,,,33
9,Jan 12 2023 9:01AM,254886,10,Emerginnova,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:53AM,254898,10,Emerginnova,1,,
1,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5,,
2,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,4,,
3,Jan 12 2023 9:27AM,254891,20,"Exact-Rx, Inc.",4,,
4,Jan 12 2023 9:18AM,254890,16,Sartorius Lab Products and Service,1,3,
5,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",,2,
6,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,24,,
7,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,33,,
8,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,33,,
9,Jan 12 2023 9:01AM,254886,10,Emerginnova,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:53AM,254898,10,Emerginnova,1,,
1,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5,,
2,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,4,,
3,Jan 12 2023 9:27AM,254891,20,"Exact-Rx, Inc.",4,,
4,Jan 12 2023 9:18AM,254890,16,Sartorius Lab Products and Service,1,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:53AM,254898,10,Emerginnova,1.0,NaN,NaN
1,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5.0,NaN,NaN
2,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,4.0,NaN,NaN
3,Jan 12 2023 9:27AM,254891,20,"Exact-Rx, Inc.",4.0,NaN,NaN
4,Jan 12 2023 9:18AM,254890,16,Sartorius Lab Products and Service,1.0,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:53AM,254898,10,Emerginnova,1.0,0.0,0.0
1,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5.0,0.0,0.0
2,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,4.0,0.0,0.0
3,Jan 12 2023 9:27AM,254891,20,"Exact-Rx, Inc.",4.0,0.0,0.0
4,Jan 12 2023 9:18AM,254890,16,Sartorius Lab Products and Service,1.0,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2803608,114.0,6.0,8.0
15,764545,115.0,0.0,33.0
16,764646,2.0,4.0,0.0
19,509731,53.0,10.0,0.0
20,254891,4.0,0.0,0.0
21,4077205,15.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2803608,114.0,6.0,8.0
1,15,764545,115.0,0.0,33.0
2,16,764646,2.0,4.0,0.0
3,19,509731,53.0,10.0,0.0
4,20,254891,4.0,0.0,0.0
5,21,4077205,15.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,114.0,6.0,8.0
1,15,115.0,0.0,33.0
2,16,2.0,4.0,0.0
3,19,53.0,10.0,0.0
4,20,4.0,0.0,0.0
5,21,15.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,114.0
1,15,Released,115.0
2,16,Released,2.0
3,19,Released,53.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,8.0,33.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,4.0,10.0,0.0,1.0
Released,114.0,115.0,2.0,53.0,4.0,15.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,8.0,33.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,4.0,10.0,0.0,1.0
2,Released,114.0,115.0,2.0,53.0,4.0,15.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,8.0,33.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,4.0,10.0,0.0,1.0
2,Released,114.0,115.0,2.0,53.0,4.0,15.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()